In [3]:
import pandas as pd
import numpy as np

In [4]:
df_raw = pd.read_excel('data/ufc_fight_details.xlsx')

In [5]:
df_raw.shape

(18945, 73)

In [6]:
df = df_raw.copy()

# Doing the thing

In [7]:
# Read the events file
events_df = pd.read_excel('data/ufc_events.xlsx')

events_df = events_df.drop_duplicates(subset='event_name')

# Merge the date information
df = df.merge(
    events_df[['event_name', 'event_date']], 
    on='event_name', 
    how='left'
)

df['event_date'] = pd.to_datetime(df['event_date'])

del events_df

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18945 entries, 0 to 18944
Data columns (total 74 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   event_name                18945 non-null  object        
 1   fight_type                18945 non-null  object        
 2   method                    18945 non-null  object        
 3   time_format               18945 non-null  object        
 4   referee                   18881 non-null  object        
 5   finish_details            7294 non-null   object        
 6   red_fighter_reach         18945 non-null  object        
 7   red_fighter_height        18945 non-null  object        
 8   red_fighter_weight        18945 non-null  object        
 9   red_fighter_stance        18903 non-null  object        
 10  red_fighter_dob           18945 non-null  object        
 11  blue_fighter_reach        18945 non-null  object        
 12  blue_fighter_heigh

In [9]:
df.head()

,event_name,fight_type,method,time_format,referee,finish_details,red_fighter_reach,red_fighter_height,red_fighter_weight,red_fighter_stance,...,red_sig_str_attempted,red_sig_str_landed,red_sig_str_pct,red_sub_att,red_td_attempted,red_td_landed,red_td_pct,red_total_str_attempted,red_total_str_landed,event_date
0,UFC 311: Makhachev vs. Moicano,UFC Lightweight Title Bout,Submission,5 Rnd (5-5-5-5-5),Herb Dean,D'Arce Choke On Ground,70,"5' 10""",155,Southpaw,...,19,6,31,1,2,1,50,31,18,2025-01-18
1,UFC 311: Makhachev vs. Moicano,UFC Bantamweight Title Bout,Decision - Unanimous,5 Rnd (5-5-5-5-5),Jason Herzog,NaN,68,"5' 6""",135,Orthodox,...,47,20,42,0,1,0,0,48,21,2025-01-18
2,UFC 311: Makhachev vs. Moicano,UFC Bantamweight Title Bout,Decision - Unanimous,5 Rnd (5-5-5-5-5),Jason Herzog,NaN,68,"5' 6""",135,Orthodox,...,49,20,40,0,3,0,0,58,27,2025-01-18
3,UFC 311: Makhachev vs. Moicano,UFC Bantamweight Title Bout,Decision - Unanimous,5 Rnd (5-5-5-5-5),Jason Herzog,NaN,68,"5' 6""",135,Orthodox,...,54,21,38,0,5,1,20,58,25,2025-01-18
4,UFC 311: Makhachev vs. Moicano,UFC Bantamweight Title Bout,Decision - Unanimous,5 Rnd (5-5-5-5-5),Jason Herzog,NaN,68,"5' 6""",135,Orthodox,...,70,36,51,0,9,4,44,82,48,2025-01-18


In [10]:
def missing(data):
# Identify missing data
    missing_data = data.isnull().sum()
    missing_percent = (data.isnull().sum() / len(data)) * 100

    # Create a DataFrame for missing data
    missing_df = pd.DataFrame({
        'Total': missing_data,
        'Percent': missing_percent
    })
    # Display columns with missing values
    return missing_df[missing_df['Total'] > 0].sort_values(by="Total", ascending=False)

missing(df)

,Total,Percent
finish_details,11651,61.499076
blue_fighter_stance,97,0.512008
referee,64,0.337820
red_fighter_stance,42,0.221694


In [11]:
df['finish_details'] = df['finish_details'].fillna(df['method'])

In [12]:
df = df.drop(columns=['referee', 'red_fighter_link', 'blue_fighter_link', 'fight_url'])


In [13]:
for i in ['blue_fighter_stance', 'red_fighter_stance']:
    df[i] = df[i].fillna(df[i].mode()[0])

missing(df)


,Total,Percent


In [14]:
# Function to convert percentage strings to float
def convert_pct_to_float(x):
    if x == '---' or pd.isna(x):
        return 0.0
    return float(x.strip('%')) / 100

# Find columns with 'pct' in their name
pct_columns = [col for col in df.columns if 'pct' in col.lower()]

# Convert each percentage column to float
for col in pct_columns:
    df[col] = df[col].apply(convert_pct_to_float)

# Verify the changes
print("Sample of converted columns:")
print(df[pct_columns].head())

Sample of converted columns:
   blue_sig_str_pct  blue_td_pct  red_sig_str_pct  red_td_pct
0              0.33         0.00             0.31        0.50
1              0.36         0.00             0.42        0.00
2              0.53         0.33             0.40        0.00
3              0.45         0.20             0.38        0.20
4              0.44         0.00             0.51        0.44


In [15]:
ctrl_columns = [col for col in df.columns if 'ctrl' in col.lower()]

# Function to convert time format to seconds
def convert_time_to_seconds(time_str):
    if pd.isna(time_str):
        return 0
    try:
        # Split minutes and seconds
        minutes, seconds = map(float, time_str.split(':'))
        return int(minutes * 60 + seconds)
    except:
        return 0

# Convert each time column to seconds
for col in ctrl_columns:
    df[col] = df[col].apply(convert_time_to_seconds)

print(df[ctrl_columns].head())

   blue_ctrl  red_ctrl
0          0        87
1          2         2
2         48        15
3         43         7
4          4        65


In [16]:
df['final_time'] = df['final_time'].apply(convert_time_to_seconds)

In [17]:
dob_columns = [col for col in df.columns if 'dob' in col.lower()]

for col in dob_columns:
    df[col] = pd.to_datetime(df[col], format='%b %d, %Y', errors='coerce')

print(df[dob_columns].head())

  red_fighter_dob blue_fighter_dob
0      1991-10-27       1989-05-21
1      1991-01-10       1996-01-03
2      1991-01-10       1996-01-03
3      1991-01-10       1996-01-03
4      1991-01-10       1996-01-03


In [18]:
# Find reach columns
reach_columns = [col for col in df.columns if 'reach' in col.lower()]

# Create a dictionary to store fighter reaches
fighter_reaches = {}

# First pass: collect all valid reaches for each fighter
for col in ['red_fighter_reach', 'blue_fighter_reach']:
    fighter_col = 'red_fighter_name' if col.startswith('red') else 'blue_fighter_name'
    
    # Get only valid reach values (not NaN or '--')
    valid_reaches = df[~df[col].isin(['--', np.nan])][[fighter_col, col]]
    
    # Add to dictionary
    for _, row in valid_reaches.iterrows():
        fighter = row[fighter_col]
        reach = row[col]
        # Handle both string and integer values
        if isinstance(reach, str):
            reach = int(reach.strip('"'))
        if fighter not in fighter_reaches:
            fighter_reaches[fighter] = reach

print(f"Number of fighters with known reach: {len(fighter_reaches)}")

# Second pass: fill in missing reaches
for col in ['red_fighter_reach', 'blue_fighter_reach']:
    fighter_col = 'red_fighter_name' if col.startswith('red') else 'blue_fighter_name'
    
    # Find rows with missing reach
    missing_mask = df[col].isin(['--', np.nan])
    
    # Fill in missing reaches where possible
    for idx in df[missing_mask].index:
        fighter = df.loc[idx, fighter_col]
        if fighter in fighter_reaches:
            df.loc[idx, col] = str(fighter_reaches[fighter]) + '"'

# Convert all reaches to integers
def convert_reach_to_int(x):
    if pd.isna(x):
        return None
    if isinstance(x, str):
        if x == '--':
            return None
        return int(x.strip('"'))
    return x

# Convert all reaches to integers where possible
for col in reach_columns:
    df[col] = df[col].apply(convert_reach_to_int)

# Calculate average reach from valid values
valid_reaches = []
for col in ['red_fighter_reach', 'blue_fighter_reach']:
    valid_reaches.extend(df[col].dropna().tolist())
    
average_reach = int(np.mean(valid_reaches))
print(f"Average reach: {average_reach} inches")

# Fill remaining missing values with the average
for col in reach_columns:
    df[col] = df[col].fillna(average_reach)

# Verify the changes
print("\nSample of final reach data:")
print(df[reach_columns].head())

# Verify no missing values remain
print("\nRemaining missing values:")
print(df[reach_columns].isna().sum())

Number of fighters with known reach: 1916
Average reach: 71 inches

Sample of final reach data:
   red_fighter_reach  blue_fighter_reach
0               70.0                72.0
1               68.0                69.0
2               68.0                69.0
3               68.0                69.0
4               68.0                69.0

Remaining missing values:
red_fighter_reach     0
blue_fighter_reach    0
dtype: int64


In [19]:
# Find weight columns
weight_columns = [col for col in df.columns if 'weight' in col.lower()]

# Create a dictionary to store fighter weights
fighter_weights = {}

# First pass: collect all valid weights for each fighter
for col in ['red_fighter_weight', 'blue_fighter_weight']:
    fighter_col = 'red_fighter_name' if col.startswith('red') else 'blue_fighter_name'
    
    # Get only valid weight values (not NaN or '--')
    valid_weights = df[~df[col].isin(['--', np.nan])][[fighter_col, col]]
    
    # Add to dictionary
    for _, row in valid_weights.iterrows():
        fighter = row[fighter_col]
        weight = row[col]
        # Handle both string and integer values
        if isinstance(weight, str):
            weight = int(weight.strip(' lbs'))
        if fighter not in fighter_weights:
            fighter_weights[fighter] = weight

print(f"Number of fighters with known weight: {len(fighter_weights)}")

# Second pass: fill in missing weights
for col in ['red_fighter_weight', 'blue_fighter_weight']:
    fighter_col = 'red_fighter_name' if col.startswith('red') else 'blue_fighter_name'
    
    # Find rows with missing weight
    missing_mask = df[col].isin(['--', np.nan])
    
    # Fill in missing weights where possible
    for idx in df[missing_mask].index:
        fighter = df.loc[idx, fighter_col]
        if fighter in fighter_weights:
            df.loc[idx, col] = str(fighter_weights[fighter]) + ' lbs'

# Convert all weights to integers
def convert_weight_to_int(x):
    if pd.isna(x):
        return None
    if isinstance(x, str):
        if x == '--':
            return None
        return int(x.strip(' lbs'))
    return x

# Convert all weights to integers where possible
for col in weight_columns:
    df[col] = df[col].apply(convert_weight_to_int)

# Calculate average weight from valid values
valid_weights = []
for col in ['red_fighter_weight', 'blue_fighter_weight']:
    valid_weights.extend(df[col].dropna().tolist())
    
average_weight = int(np.mean(valid_weights))
print(f"Average weight: {average_weight} lbs")

# Fill remaining missing values with the average
for col in weight_columns:
    df[col] = df[col].fillna(average_weight)

# Verify the changes
print("\nSample of final weight data:")
print(df[weight_columns].head())

# Verify no missing values remain
print("\nRemaining missing values:")
print(df[weight_columns].isna().sum())

Number of fighters with known weight: 2544
Average weight: 165 lbs

Sample of final weight data:
   red_fighter_weight  blue_fighter_weight
0               155.0                155.0
1               135.0                135.0
2               135.0                135.0
3               135.0                135.0
4               135.0                135.0

Remaining missing values:
red_fighter_weight     0
blue_fighter_weight    0
dtype: int64


In [20]:
# Find height columns
height_columns = [col for col in df.columns if 'height' in col.lower()]

# Function to convert height to centimeters
def convert_height_to_cm(height_str):
    if pd.isna(height_str) or height_str == '--':
        return None
    try:
        # Split the feet and inches parts
        feet_str, inches_str = height_str.split("'")
        feet = int(feet_str)
        inches = int(inches_str.strip('"'))
        
        # Convert to centimeters
        # 1 foot = 30.48 cm
        # 1 inch = 2.54 cm
        total_cm = (feet * 30.48) + (inches * 2.54)
        return round(total_cm)
    except:
        return None

# Create a dictionary to store fighter heights
fighter_heights = {}

# First pass: collect all valid heights for each fighter
for col in ['red_fighter_height', 'blue_fighter_height']:
    fighter_col = 'red_fighter_name' if col.startswith('red') else 'blue_fighter_name'
    
    # Get only valid height values
    valid_heights = df[~df[col].isin(['--', np.nan])][[fighter_col, col]]
    
    # Add to dictionary
    for _, row in valid_heights.iterrows():
        fighter = row[fighter_col]
        height = convert_height_to_cm(row[col])
        if height and fighter not in fighter_heights:
            fighter_heights[fighter] = height

print(f"Number of fighters with known height: {len(fighter_heights)}")

# Second pass: fill in missing heights
for col in ['red_fighter_height', 'blue_fighter_height']:
    fighter_col = 'red_fighter_name' if col.startswith('red') else 'blue_fighter_name'
    
    # Convert all heights to cm
    df[col] = df[col].apply(convert_height_to_cm)
    
    # Fill missing heights from fighter dictionary
    for idx in df[df[col].isna()].index:
        fighter = df.loc[idx, fighter_col]
        if fighter in fighter_heights:
            df.loc[idx, col] = fighter_heights[fighter]

# Calculate average height from valid values
valid_heights = []
for col in height_columns:
    valid_heights.extend(df[col].dropna().tolist())
    
average_height = round(np.mean(valid_heights))
print(f"Average height: {average_height} cm")

# Fill remaining missing values with the average
for col in height_columns:
    df[col] = df[col].fillna(average_height)

# Verify the changes
print("\nSample of final height data (in cm):")
print(df[height_columns].head())

# Verify no missing values remain
print("\nRemaining missing values:")
print(df[height_columns].isna().sum())

Number of fighters with known height: 2541
Average height: 178 cm

Sample of final height data (in cm):
   red_fighter_height  blue_fighter_height
0               178.0                180.0
1               168.0                173.0
2               168.0                173.0
3               168.0                173.0
4               168.0                173.0

Remaining missing values:
red_fighter_height     0
blue_fighter_height    0
dtype: int64


In [21]:
# Count unique fight types
fight_type_counts = df['fight_type'].value_counts()

print("Unique fight types and their counts:")
print(fight_type_counts)

print(f"\nTotal number of unique fight types: {len(fight_type_counts)}")

Unique fight types and their counts:
fight_type
Lightweight Bout               3040
Welterweight Bout              2941
Middleweight Bout              2246
Featherweight Bout             1847
Bantamweight Bout              1649
                               ... 
UFC 6 Tournament Title Bout       1
UFC 5 Tournament Title Bout       1
UFC 4 Tournament Title Bout       1
UFC 3 Tournament Title Bout       1
UFC 2 Tournament Title Bout       1
Name: count, Length: 113, dtype: int64

Total number of unique fight types: 113


In [22]:
# Count unique fight types
finish_type_counts = df['finish_details'].value_counts()

print("Unique fight types and their counts:")
print(finish_type_counts)

print(f"\nTotal number of unique fight types: {len(finish_type_counts)}")

Unique fight types and their counts:
finish_details
Decision - Unanimous                                     8909
Decision - Split                                         2433
Punch to Head At Distance                                1043
Rear Naked Choke                                         1036
Punches to Head At Distance                               565
                                                         ... 
Elbows to Head On Ground\n      Submission to Strikes       1
Other - Choke From Side Control                             1
Gi Choke From Back Control                                  1
Ezekiel Choke From Side Control                             1
Other  \n      Fatigue                                      1
Name: count, Length: 470, dtype: int64

Total number of unique fight types: 470


In [23]:
# Count unique fight types
method_counts = df['method'].value_counts()

print("Unique fight types and their counts:")
print(method_counts)

print(f"\nTotal number of unique fight types: {len(method_counts)}")

Unique fight types and their counts:
method
Decision - Unanimous       8997
KO/TKO                     4169
Submission                 2636
Decision - Split           2448
Decision - Majority         299
TKO - Doctor's Stoppage     187
Overturned                  124
Could Not Continue           44
DQ                           37
Other                         4
Name: count, dtype: int64

Total number of unique fight types: 10


In [24]:
df = df.drop(columns=['fight_type', # weight class, we will create a new column for this based on the weights
                      'event_name',
                      'finish_details',
                      ])


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18945 entries, 0 to 18944
Data columns (total 67 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   method                    18945 non-null  object        
 1   time_format               18945 non-null  object        
 2   red_fighter_reach         18945 non-null  float64       
 3   red_fighter_height        18945 non-null  float64       
 4   red_fighter_weight        18945 non-null  float64       
 5   red_fighter_stance        18945 non-null  object        
 6   red_fighter_dob           18869 non-null  datetime64[ns]
 7   blue_fighter_reach        18945 non-null  float64       
 8   blue_fighter_height       18945 non-null  float64       
 9   blue_fighter_weight       18945 non-null  float64       
 10  blue_fighter_stance       18945 non-null  object        
 11  blue_fighter_dob          18728 non-null  datetime64[ns]
 12  round             

In [26]:
missing(df)

,Total,Percent
blue_fighter_dob,217,1.145421
red_fighter_dob,76,0.401161


In [27]:
for i in dob_columns:
    df[i] = df[i].fillna(df[i].mean())

missing(df)

,Total,Percent


In [28]:
# Count unique fight types
blue_wins = df['blue_fighter_status'].value_counts()

print("Unique fight types and their counts:")
print(blue_wins)

print(f"\nTotal number of unique fight types: {len(blue_wins)}")

Unique fight types and their counts:
blue_fighter_status
L     11920
W      6670
D       187
NC      168
Name: count, dtype: int64

Total number of unique fight types: 4


In [29]:
red_wins = df['red_fighter_status'].value_counts()

print("Unique fight types and their counts:")
print(red_wins)

print(f"\nTotal number of unique fight types: {len(red_wins)}")


Unique fight types and their counts:
red_fighter_status
W     11920
L      6670
D       187
NC      168
Name: count, dtype: int64

Total number of unique fight types: 4


-   W: wins
-   L: losses
-   D: draws
-   NC: no contest



In [30]:
# Identify columns ending with '_status'
status_columns = [col for col in df.columns if col.endswith('_status')]

# Create a mask to filter out rows where any status column has 'D' or 'NC'
mask = df[status_columns].isin(['D', 'NC']).any(axis=1)

# Drop the rows that match the mask
df = df[~mask]

# Verify the changes
print(f"Total number of rows after dropping statuses 'D' or 'NC': {len(df)}")

del mask, missing_mask

Total number of rows after dropping statuses 'D' or 'NC': 18590


In [31]:
# Create a new column for red fighter win status
df.insert(0, 'red_fighter_win', df['red_fighter_status'].apply(lambda x: 1 if x == 'W' else 0))

# Drop columns ending with '_status'
df = df.drop(columns=status_columns)

In [32]:
def get_weight_class(weight_lbs):
    if weight_lbs <= 125:
        return 'Flyweight'
    elif weight_lbs <= 135:
        return 'Bantamweight'
    elif weight_lbs <= 145:
        return 'Featherweight'
    elif weight_lbs <= 155:
        return 'Lightweight'
    elif weight_lbs <= 170:
        return 'Welterweight'
    elif weight_lbs <= 185:
        return 'Middleweight'
    elif weight_lbs <= 205:
        return 'Light Heavyweight'
    else:
        return 'Heavyweight'

# Add weight class columns
df['red_fighter_weight_class'] = df['red_fighter_weight'].apply(get_weight_class)
df['blue_fighter_weight_class'] = df['blue_fighter_weight'].apply(get_weight_class)

# Verify that both fighters are in the same weight class
df['weight_class_match'] = df['red_fighter_weight_class'] == df['blue_fighter_weight_class']

# Print any mismatches
mismatches = df[~df['weight_class_match']]
if len(mismatches) > 0:
    print(f"\nFound {len(mismatches)} fights with mismatched weight classes:")
    print(mismatches[['red_fighter_name', 'red_fighter_weight', 'red_fighter_weight_class',
                      'blue_fighter_name', 'blue_fighter_weight', 'blue_fighter_weight_class']])

# Create a single weight_class column since both fighters should be in the same class
df['weight_class'] = df['red_fighter_weight_class']

# Drop the individual weight class columns and the match verification column
df = df.drop(columns=['red_fighter_weight_class', 'blue_fighter_weight_class', 'weight_class_match'])
# # Convert weight_class to one-hot encoded columns
# df = pd.get_dummies(df, columns=['weight_class'], prefix='weight_class')


Found 5052 fights with mismatched weight classes:
        red_fighter_name  red_fighter_weight red_fighter_weight_class  \
9        Jailton Almeida               205.0        Light Heavyweight   
131    Kennedy Nzechukwu               205.0        Light Heavyweight   
146       Zhang Mingyang               205.0        Light Heavyweight   
187          David Onama               145.0            Featherweight   
188          David Onama               145.0            Featherweight   
...                  ...                 ...                      ...   
18921    Shara Magomedov               185.0             Middleweight   
18922    Shara Magomedov               185.0             Middleweight   
18942   Hamdy Abdelwahab               264.0              Heavyweight   
18943   Hamdy Abdelwahab               264.0              Heavyweight   
18944   Hamdy Abdelwahab               264.0              Heavyweight   

      blue_fighter_name  blue_fighter_weight blue_fighter_weight_class  

In [33]:
# Calculate age for both red and blue fighters
df['red_fighter_age'] = (pd.to_datetime(df['event_date']) - pd.to_datetime(df['red_fighter_dob'])).dt.total_seconds() / (365.25 * 24 * 60 * 60)
df['blue_fighter_age'] = (pd.to_datetime(df['event_date']) - pd.to_datetime(df['blue_fighter_dob'])).dt.total_seconds() / (365.25 * 24 * 60 * 60)

# Round ages to 1 decimal place
df['red_fighter_age'] = df['red_fighter_age'].round(1)
df['blue_fighter_age'] = df['blue_fighter_age'].round(1)

In [34]:
# Define the order of columns
red_winner_columns = ['red_fighter_win']  # Start with the red winner column
fighter_name_columns = ['red_fighter_name', 'blue_fighter_name']  # Fighter names
non_fighter_columns = [col for col in df.columns if not (col.startswith('red') or col.startswith('blue')) and col != 'red_fighter_win']
red_columns = [col for col in df.columns if col.startswith('red') and col != 'red_fighter_win' and col != 'red_fighter_name']
blue_columns = [col for col in df.columns if col.startswith('blue') and col != 'blue_fighter_name']

# Combine all columns in the desired order
ordered_columns = red_winner_columns + fighter_name_columns + non_fighter_columns + red_columns + blue_columns

# Reorder the DataFrame
df = df[ordered_columns]

In [35]:
# Identify numeric columns
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns

# Filter out columns that end with 'pct'
columns_to_convert = [col for col in numeric_columns if not col.endswith('pct')]

# Convert the selected columns to integers
df[columns_to_convert] = df[columns_to_convert].astype(int)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18590 entries, 0 to 18944
Data columns (total 69 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   red_fighter_win           18590 non-null  int64         
 1   red_fighter_name          18590 non-null  object        
 2   blue_fighter_name         18590 non-null  object        
 3   method                    18590 non-null  object        
 4   time_format               18590 non-null  object        
 5   round                     18590 non-null  int64         
 6   final_time                18590 non-null  int64         
 7   event_date                18590 non-null  datetime64[ns]
 8   weight_class              18590 non-null  object        
 9   red_fighter_reach         18590 non-null  int64         
 10  red_fighter_height        18590 non-null  int64         
 11  red_fighter_weight        18590 non-null  int64         
 12  red_fighter_stance     

In [37]:
missing(df)

,Total,Percent


# Add trailing data

In [38]:
# Calculate Trailing Averages for Fighter Statistics

# First, ensure the data is sorted by event date
df = df.sort_values(by=['event_date', 'red_fighter_name', 'blue_fighter_name', 'round'])

# Identify columns to calculate trailing averages for (only numeric columns)
red_stat_cols = [col for col in df.columns if col.startswith('red_') and 
                not col.startswith('red_fighter_') and 
                pd.api.types.is_numeric_dtype(df[col])]

blue_stat_cols = [col for col in df.columns if col.startswith('blue_') and 
                 not col.startswith('blue_fighter_') and 
                 pd.api.types.is_numeric_dtype(df[col])]

print(f"Found {len(red_stat_cols)} red stats and {len(blue_stat_cols)} blue stats to process")

Found 24 red stats and 24 blue stats to process


In [39]:
red_stat_cols

['red_body_attempted',
 'red_body_landed',
 'red_clinch_attempted',
 'red_clinch_landed',
 'red_ctrl',
 'red_distance_attempted',
 'red_distance_landed',
 'red_ground_attempted',
 'red_ground_landed',
 'red_head_attempted',
 'red_head_landed',
 'red_kd',
 'red_leg_attempted',
 'red_leg_landed',
 'red_rev',
 'red_sig_str_attempted',
 'red_sig_str_landed',
 'red_sig_str_pct',
 'red_sub_att',
 'red_td_attempted',
 'red_td_landed',
 'red_td_pct',
 'red_total_str_attempted',
 'red_total_str_landed']

In [40]:
all_performances = []

# Process each row to extract fighter performances
for idx, row in df.iterrows():
    event_date = row['event_date']
    fight_id = idx  # Use row index as a unique fight identifier
    
    # Extract red fighter stats
    red_fighter = row['red_fighter_name']
    for col in red_stat_cols:
        base_stat = col[4:]  # Remove 'red_' prefix
        if pd.notnull(row[col]):
            all_performances.append({
                'fighter': red_fighter,
                'event_date': event_date,
                'fight_id': fight_id,
                'stat': base_stat,
                'value': row[col]
            })
    
    # Extract blue fighter stats
    blue_fighter = row['blue_fighter_name']
    for col in blue_stat_cols:
        base_stat = col[5:]  # Remove 'blue_' prefix
        if pd.notnull(row[col]):
            all_performances.append({
                'fighter': blue_fighter,
                'event_date': event_date,
                'fight_id': fight_id,
                'stat': base_stat,
                'value': row[col]
            })

# Convert to DataFrame
perf_df = pd.DataFrame(all_performances)
perf_df = perf_df.sort_values(by=['fighter', 'stat', 'event_date'])

print("Calculating weighted trailing averages...")

# Function to calculate weighted average based on recency
def weighted_trailing_average(group):
    result = pd.Series(index=group.index)
    
    for i in range(len(group)):
        if i == 0:
            # First fight has no trailing average
            result.iloc[i] = None
        else:
            # Get previous fights
            prev_fights = group.iloc[:i]
            
            # Calculate weights based on recency
            # More recent fights get higher weights
            # You can adjust the decay factor to control how quickly weights diminish
            # A value closer to 1 (e.g., 0.9) means older fights still have significant weight
            # A value closer to 0 (e.g., 0.5) means recent fights are much more important than older ones
            decay_factor = 0.5  # Adjust this value between 0 and 1
            weights = np.power(decay_factor, np.arange(len(prev_fights)-1, -1, -1))
            
            # Normalize weights to sum to 1
            weights = weights / weights.sum()
            
            # Calculate weighted average
            weighted_avg = (prev_fights['value'] * weights).sum()
            result.iloc[i] = weighted_avg
            
    return result

# Apply weighted trailing average to each fighter and stat group
perf_df['weighted_trailing_avg'] = perf_df.groupby(['fighter', 'stat']).apply(
    lambda x: weighted_trailing_average(x)
).reset_index(level=[0, 1], drop=True)

# Pivot back to wide format for mapping to original dataframe
trailing_stats = perf_df.pivot_table(
    index=['fighter', 'event_date', 'fight_id'],
    columns='stat',
    values='weighted_trailing_avg'
).reset_index()

del perf_df

print("Mapping weighted trailing averages back to main dataframe...")

# Create a dictionary to store all new trailing columns
trailing_values = {}
for col in red_stat_cols:
    base_stat = col[4:]
    trailing_values[f"{col}_weighted_trailing"] = [None] * len(df)

for col in blue_stat_cols:
    base_stat = col[5:]
    trailing_values[f"{col}_weighted_trailing"] = [None] * len(df)

# For each row in the original dataframe, map the trailing averages
for i, (idx, row) in enumerate(df.iterrows()):
    # Map red fighter trailing stats
    red_fighter = row['red_fighter_name']
    red_stats = trailing_stats[
        (trailing_stats['fighter'] == red_fighter) & 
        (trailing_stats['event_date'] < row['event_date'])
    ]
    
    if not red_stats.empty:
        latest_red_stats = red_stats.iloc[-1]
        for col in red_stat_cols:
            base_stat = col[4:]
            if base_stat in latest_red_stats and pd.notnull(latest_red_stats[base_stat]):
                trailing_values[f"{col}_weighted_trailing"][i] = latest_red_stats[base_stat]
    
    # Map blue fighter trailing stats
    blue_fighter = row['blue_fighter_name']
    blue_stats = trailing_stats[
        (trailing_stats['fighter'] == blue_fighter) & 
        (trailing_stats['event_date'] < row['event_date'])
    ]
    
    if not blue_stats.empty:
        latest_blue_stats = blue_stats.iloc[-1]
        for col in blue_stat_cols:
            base_stat = col[5:]
            if base_stat in latest_blue_stats and pd.notnull(latest_blue_stats[base_stat]):
                trailing_values[f"{col}_weighted_trailing"][i] = latest_blue_stats[base_stat]

# Add all trailing columns at once
trailing_df = pd.DataFrame(trailing_values, index=df.index)
df = pd.concat([df, trailing_df], axis=1)

# Fill NaN values with 0
weighted_trailing_cols = [col for col in df.columns if 'weighted_trailing' in col]
df[weighted_trailing_cols] = df[weighted_trailing_cols].fillna(0)

del all_performances, trailing_df, trailing_stats

Calculating weighted trailing averages...


C:\Users\hegyi\AppData\Local\Temp\ipykernel_3144\1121071972.py:70: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  perf_df['weighted_trailing_avg'] = perf_df.groupby(['fighter', 'stat']).apply(


Mapping weighted trailing averages back to main dataframe...


In [41]:
# Display a sample of the trailing statistics
trailing_cols = [col for col in df.columns if col.endswith('_trailing')]
print(f"Created {len(trailing_cols)} trailing statistic columns")
print("\nSample of trailing statistics:")

Created 48 trailing statistic columns

Sample of trailing statistics:


In [42]:
df[['red_fighter_name', 'blue_fighter_name', 'event_date'] + trailing_cols[:5]].tail()

,red_fighter_name,blue_fighter_name,event_date,red_body_attempted_weighted_trailing,red_body_landed_weighted_trailing,red_clinch_attempted_weighted_trailing,red_clinch_landed_weighted_trailing,red_ctrl_weighted_trailing
18788,Justin Gaethje,Rafael Fiziev,2025-03-08,12.455157,8.452134,0.193364,0.153324,0.083145
18793,King Green,Mauricio Ruffy,2025-03-08,16.721742,12.701791,3.506415,3.318907,29.276953
18800,Mairon Santos,Francis Marshall,2025-03-08,6.000000,6.000000,2.000000,1.000000,0.000000
18801,Mairon Santos,Francis Marshall,2025-03-08,6.000000,6.000000,2.000000,1.000000,0.000000
18802,Mairon Santos,Francis Marshall,2025-03-08,6.000000,6.000000,2.000000,1.000000,0.000000


In [43]:
# Fill NaN values with 0 in all trailing columns
df[trailing_cols] = df[trailing_cols].fillna(0)

In [44]:
missing(df)

,Total,Percent


In [45]:
# Calculate days since last fight for each fighter

print("Calculating days since last fight...")

# Create a dictionary to track each fighter's last fight date
fighter_last_fight_dates = {}

# Create new columns for days since last fight
df['red_fighter_days_since_last'] = 0  # Default to 0
df['blue_fighter_days_since_last'] = 0  # Default to 0

# Sort by event date to process fights chronologically
df_sorted = df.sort_values(by='event_date')

# Process each fight to calculate days since last fight
for idx, row in df_sorted.iterrows():
    event_date = row['event_date']
    red_fighter = row['red_fighter_name']
    blue_fighter = row['blue_fighter_name']
    
    # Calculate days since last fight for red fighter
    if red_fighter in fighter_last_fight_dates:
        last_fight_date = fighter_last_fight_dates[red_fighter]
        days_since_last = (event_date - last_fight_date).days
        df.at[idx, 'red_fighter_days_since_last'] = days_since_last
    # else: It's the fighter's first recorded match, keep the default 0
    
    # Calculate days since last fight for blue fighter
    if blue_fighter in fighter_last_fight_dates:
        last_fight_date = fighter_last_fight_dates[blue_fighter]
        days_since_last = (event_date - last_fight_date).days
        df.at[idx, 'blue_fighter_days_since_last'] = days_since_last
    # else: It's the fighter's first recorded match, keep the default 0
    
    # Update the last fight date for both fighters
    fighter_last_fight_dates[red_fighter] = event_date
    fighter_last_fight_dates[blue_fighter] = event_date

# Check for any negative values (which would indicate a data issue)
red_negative = (df['red_fighter_days_since_last'] < 0).sum()
blue_negative = (df['blue_fighter_days_since_last'] < 0).sum()
print(f"\nNegative values check - Red: {red_negative}, Blue: {blue_negative}")

del df_sorted

Calculating days since last fight...

Negative values check - Red: 0, Blue: 0


In [46]:
missing(df)

,Total,Percent


# Add trailing to test

In [47]:
test_df = pd.read_excel("data/ufc_upcoming_events.xlsx")


In [48]:
test = test_df.copy()


In [49]:
def convert_dates_in_test_set(test_df):
    """
    Convert date strings in format 'Month Day, Year' to ISO format 'YYYY-MM-DD'
    
    Parameters:
    test_df - Test dataframe with event_date column in string format
    
    Returns:
    test_df with converted event_date column
    """
    import pandas as pd
    from datetime import datetime
    
    # Make a copy to avoid modifying the original if needed
    df_copy = test_df.copy()
    
    # Define a function to convert a single date string
    def convert_date_string(date_str):
        try:
            # Parse the date string
            date_obj = datetime.strptime(date_str, '%B %d, %Y')
            # Convert to ISO format
            return date_obj.strftime('%Y-%m-%d')
        except Exception as e:
            print(f"Error converting date '{date_str}': {e}")
            return date_str  # Return original if conversion fails
    
    # Check if event_date is already a datetime object
    if pd.api.types.is_datetime64_any_dtype(df_copy['event_date']):
        print("event_date is already in datetime format, converting to ISO string format")
        df_copy['event_date'] = df_copy['event_date'].dt.strftime('%Y-%m-%d')
    else:
        # Apply the conversion function to the event_date column
        print("Converting event_date from string to ISO format")
        df_copy['event_date'] = df_copy['event_date'].apply(convert_date_string)
    
    # Convert the ISO format strings to datetime objects for proper date handling
    df_copy['event_date'] = pd.to_datetime(df_copy['event_date'])
    
    print(f"Converted {len(df_copy)} dates")
    print(f"Sample of converted dates: {df_copy['event_date'].head()}")
    
    return df_copy

# Usage example:
test = convert_dates_in_test_set(test)

Converting event_date from string to ISO format
Converted 13 dates
Sample of converted dates: 0   2025-03-15
1   2025-03-15
2   2025-03-15
3   2025-03-15
4   2025-03-15
Name: event_date, dtype: datetime64[ns]


In [50]:
df.columns


Index(['red_fighter_win', 'red_fighter_name', 'blue_fighter_name', 'method',
       'time_format', 'round', 'final_time', 'event_date', 'weight_class',
       'red_fighter_reach',
       ...
       'blue_sig_str_landed_weighted_trailing',
       'blue_sig_str_pct_weighted_trailing', 'blue_sub_att_weighted_trailing',
       'blue_td_attempted_weighted_trailing',
       'blue_td_landed_weighted_trailing', 'blue_td_pct_weighted_trailing',
       'blue_total_str_attempted_weighted_trailing',
       'blue_total_str_landed_weighted_trailing',
       'red_fighter_days_since_last', 'blue_fighter_days_since_last'],
      dtype='object', length=119)

In [51]:
# Get a list of all columns that contain 'fighter1'
fighter1_cols = [col for col in test.columns if 'fighter1' in col]

# Create a dictionary mapping old column names to new column names
rename_dict = {col: col.replace('fighter1', 'red_fighter') for col in fighter1_cols}

# Rename the columns
test = test.rename(columns=rename_dict)

# Get a list of all columns that contain 'fighter1'
fighter2_cols = [col for col in test.columns if 'fighter2' in col]

# Create a dictionary mapping old column names to new column names
rename_dict = {col: col.replace('fighter2', 'blue_fighter') for col in fighter2_cols}

# Rename the columns
test = test.rename(columns=rename_dict)
test = test.rename(columns = {"red_fighter":"red_fighter_name", "blue_fighter":"blue_fighter_name"})

In [52]:
# Assuming your DataFrame is called df
# This will remove both "Bout" and "Women's" from the weight_class column
test['weight_class'] = test['weight_class'].str.replace('Bout', '').str.strip()

In [53]:
upcoming_processed = test.copy()

def convert_time_to_seconds(time_str):
    if pd.isna(time_str):
        return 0
    try:
        # Split minutes and seconds
        minutes, seconds = map(float, time_str.split(':'))
        return float(minutes * 60 + seconds)
    except:
        return 0

def convert_height_to_cm(height_str):
    if pd.isna(height_str) or height_str == '--':
        return None
    try:
        # Split the feet and inches parts
        feet_str, inches_str = height_str.split("'")
        feet = int(feet_str)
        inches = int(inches_str.strip('"'))
        
        # Convert to centimeters
        total_cm = (feet * 30.48) + (inches * 2.54)
        return round(total_cm)
    except:
        return None

def convert_reach_to_cm(reach_str):
    if pd.isna(reach_str) or reach_str == '--':
        return None
    try:
        # Convert inches to cm
        inches = int(reach_str.strip('"'))
        return inches
    except:
        return None

def convert_weight_to_kg(weight_str):
    if pd.isna(weight_str) or weight_str == '--':
        return None
    try:
        # Convert lbs to kg
        lbs = int(weight_str.strip(' lbs.'))
        return lbs
    except:
        return None

def convert_percentage(pct_str):
    if pd.isna(pct_str):
        return 0.0
    try:
        return float(pct_str.strip('%')) / 100
    except:
        return 0.0

def extract_record(record_str):
    if pd.isna(record_str):
        return 0, 0
    try:
        parts = record_str.split('-')
        return int(parts[0]), int(parts[1])
    except:
        return 0, 0

# Process each fighter's data
for prefix in ['red_fighter', 'blue_fighter']:
    # Convert fight time to seconds
    upcoming_processed[f'{prefix}_avg_fight_time'] = upcoming_processed[f'{prefix}_average_fight_time'].apply(convert_time_to_seconds)
    
    # Convert height to cm
    upcoming_processed[f'{prefix}_height'] = upcoming_processed[f'{prefix}_height'].apply(convert_height_to_cm)
    
    # Convert reach to cm
    upcoming_processed[f'{prefix}_reach'] = upcoming_processed[f'{prefix}_reach'].apply(convert_reach_to_cm)
    
    # Convert weight to kg
    upcoming_processed[f'{prefix}_weight'] = upcoming_processed[f'{prefix}_weight'].apply(convert_weight_to_kg)
    upcoming_processed[f'{prefix}_dob'] = pd.to_datetime(upcoming_processed[f'{prefix}_dob'], format='%b %d, %Y')

    
    # Convert percentages
    for stat in ['striking_accuracy', 'takedown_accuracy', 'defense', 'takedown_defense']:
        col = f'{prefix}_{stat}'
        if col in upcoming_processed.columns:
            upcoming_processed[col] = upcoming_processed[col].apply(convert_percentage)
    
    # Extract wins and losses
    wins_losses = upcoming_processed[f'{prefix}_wins/losses/draws'].apply(extract_record)
    upcoming_processed[f'{prefix}_wins'] = [x[0] for x in wins_losses]
    upcoming_processed[f'{prefix}_losses'] = [x[1] for x in wins_losses]

# Rename columns to match our calculated statistics format
column_mapping = {
    'strikes_absorbed_per_min_(sapm)': 'strikes_absorbed_per_min',
    'strikes_landed_per_min_(slpm)': 'strikes_landed_per_min',
    'submission_average/15_min': 'submission_per_15',
    'takedowns_average/15_min': 'takedowns_per_15'
}

for old_suffix, new_suffix in column_mapping.items():
    for prefix in ['red_fighter', 'blue_fighter']:
        old_col = f'{prefix}_{old_suffix}'
        new_col = f'{prefix}_{new_suffix}'
        if old_col in upcoming_processed.columns:
            upcoming_processed = upcoming_processed.rename(columns={old_col: new_col})

# Drop unnecessary columns
cols_to_drop = [
    'event_name', 'event_location', 'event_link',
    'fight_link', 'red_fighter_link', 'blue_fighter_link',
    'red_fighter_nickname', 'blue_fighter_nickname',
    'red_fighter_wins/losses/draws', 'blue_fighter_wins/losses/draws',
    'red_fighter_average_fight_time', 'blue_fighter_average_fight_time'  # because we created avg_fight_time
]
upcoming_processed = upcoming_processed.drop(columns=cols_to_drop)

test = upcoming_processed.copy()
del upcoming_processed

In [56]:
def enrich_test_data(df, test_df):
    """
    Add weighted trailing averages and days since last fight to test set.
    
    Parameters:
    df - Full training dataset with fight history
    test_df - Test dataset that needs enrichment
    
    Returns:
    test_df with added features
    """
    # Define stat columns (excluding any weighted_trailing columns)
    red_stat_cols = [col for col in df.columns if col.startswith('red_') and 
                     not col.startswith('red_fighter') and 
                     'weighted_trailing' not in col]
    
    blue_stat_cols = [col for col in df.columns if col.startswith('blue_') and 
                      not col.startswith('blue_fighter') and 
                      'weighted_trailing' not in col]
    
    print(f"Processing {len(red_stat_cols)} red stats and {len(blue_stat_cols)} blue stats")
    
    # Create a list to store all fighter performances
    all_performances = []
    
    # Process each row in the full dataset to extract fighter performances
    for idx, row in df.iterrows():
        event_date = row['event_date']
        fight_id = idx  # Use row index as a unique fight identifier
        
        # Extract red fighter stats
        red_fighter = row['red_fighter_name']
        for col in red_stat_cols:
            base_stat = col[4:]  # Remove 'red_' prefix
            if pd.notnull(row[col]):
                all_performances.append({
                    'fighter': red_fighter,
                    'event_date': event_date,
                    'fight_id': fight_id,
                    'stat': base_stat,
                    'value': row[col]
                })
        
        # Extract blue fighter stats
        blue_fighter = row['blue_fighter_name']
        for col in blue_stat_cols:
            base_stat = col[5:]  # Remove 'blue_' prefix
            if pd.notnull(row[col]):
                all_performances.append({
                    'fighter': blue_fighter,
                    'event_date': event_date,
                    'fight_id': fight_id,
                    'stat': base_stat,
                    'value': row[col]
                })
    
    # Convert to DataFrame
    perf_df = pd.DataFrame(all_performances)
    perf_df = perf_df.sort_values(by=['fighter', 'stat', 'event_date'])
    
    print("Calculating weighted trailing averages...")
    
    # Function to calculate weighted average based on recency
    def weighted_trailing_average(group):
        result = pd.Series(index=group.index)
        
        for i in range(len(group)):
            if i == 0:
                # First fight has no trailing average
                result.iloc[i] = None
            else:
                # Get previous fights
                prev_fights = group.iloc[:i]
                
                # Calculate weights based on recency
                decay_factor = 0.5  # Same decay factor as in original code
                weights = np.power(decay_factor, np.arange(len(prev_fights)-1, -1, -1))
                
                # Normalize weights to sum to 1
                weights = weights / weights.sum()
                
                # Calculate weighted average
                weighted_avg = (prev_fights['value'] * weights).sum()
                result.iloc[i] = weighted_avg
                
        return result
    
    # Apply weighted trailing average to each fighter and stat group
    perf_df['weighted_trailing_avg'] = perf_df.groupby(['fighter', 'stat']).apply(
        lambda x: weighted_trailing_average(x)
    ).reset_index(level=[0, 1], drop=True)
    
    # Pivot back to wide format for mapping to test dataframe
    trailing_stats = perf_df.pivot_table(
        index=['fighter', 'event_date', 'fight_id'],
        columns='stat',
        values='weighted_trailing_avg'
    ).reset_index()
    
    print("Mapping weighted trailing averages to test dataframe...")
    
    # Create a dictionary to store all new trailing columns for test set
    trailing_values = {}
    for col in red_stat_cols:
        base_stat = col[4:]
        trailing_values[f"{col}_weighted_trailing"] = [None] * len(test_df)
    
    for col in blue_stat_cols:
        base_stat = col[5:]
        trailing_values[f"{col}_weighted_trailing"] = [None] * len(test_df)
    
    # For each row in the test dataframe, map the trailing averages
    for i, (idx, row) in enumerate(test_df.iterrows()):
        # Map red fighter trailing stats
        red_fighter = row['red_fighter_name']
        red_stats = trailing_stats[
            (trailing_stats['fighter'] == red_fighter) & 
            (trailing_stats['event_date'] < row['event_date'])
        ]
        
        if not red_stats.empty:
            latest_red_stats = red_stats.iloc[-1]
            for col in red_stat_cols:
                base_stat = col[4:]
                if base_stat in latest_red_stats and pd.notnull(latest_red_stats[base_stat]):
                    trailing_values[f"{col}_weighted_trailing"][i] = latest_red_stats[base_stat]
        
        # Map blue fighter trailing stats
        blue_fighter = row['blue_fighter_name']
        blue_stats = trailing_stats[
            (trailing_stats['fighter'] == blue_fighter) & 
            (trailing_stats['event_date'] < row['event_date'])
        ]
        
        if not blue_stats.empty:
            latest_blue_stats = blue_stats.iloc[-1]
            for col in blue_stat_cols:
                base_stat = col[5:]
                if base_stat in latest_blue_stats and pd.notnull(latest_blue_stats[base_stat]):
                    trailing_values[f"{col}_weighted_trailing"][i] = latest_blue_stats[base_stat]
    
    # Add all trailing columns at once
    trailing_df = pd.DataFrame(trailing_values, index=test_df.index)
    test_df = pd.concat([test_df, trailing_df], axis=1)
    
    # Fill NaN values with 0
    weighted_trailing_cols = [col for col in test_df.columns if 'weighted_trailing' in col]
    test_df[weighted_trailing_cols] = test_df[weighted_trailing_cols].fillna(0)
    
    print("Calculating days since last fight for test set...")
    
    # Create a dictionary to track each fighter's last fight date
    # Initialize with data from the training set
    fighter_last_fight_dates = {}
    
    # Sort the full dataset by event date to process fights chronologically
    df_sorted = df.sort_values(by='event_date')
    
    # Process each fight in the training set to build fighter history
    for idx, row in df_sorted.iterrows():
        event_date = row['event_date']
        red_fighter = row['red_fighter_name']
        blue_fighter = row['blue_fighter_name']
        
        # Update the last fight date for both fighters
        fighter_last_fight_dates[red_fighter] = event_date
        fighter_last_fight_dates[blue_fighter] = event_date
    
    # Create new columns for days since last fight in test set
    test_df['red_fighter_days_since_last'] = 0  # Default to 0
    test_df['blue_fighter_days_since_last'] = 0  # Default to 0
    
    # Sort test set by event date
    test_df_sorted = test_df.sort_values(by='event_date')
    
    # Process each fight in the test set
    for idx, row in test_df_sorted.iterrows():
        event_date = row['event_date']
        red_fighter = row['red_fighter_name']
        blue_fighter = row['blue_fighter_name']
        
        # Calculate days since last fight for red fighter
        if red_fighter in fighter_last_fight_dates:
            last_fight_date = fighter_last_fight_dates[red_fighter]
            days_since_last = (event_date - last_fight_date).days
            test_df.at[idx, 'red_fighter_days_since_last'] = days_since_last
        # else: It's the fighter's first recorded match, keep the default 0
        
        # Calculate days since last fight for blue fighter
        if blue_fighter in fighter_last_fight_dates:
            last_fight_date = fighter_last_fight_dates[blue_fighter]
            days_since_last = (event_date - last_fight_date).days
            test_df.at[idx, 'blue_fighter_days_since_last'] = days_since_last
        # else: It's the fighter's first recorded match, keep the default 0
        
        # Update the last fight date for both fighters
        fighter_last_fight_dates[red_fighter] = event_date
        fighter_last_fight_dates[blue_fighter] = event_date
    
    # Check for any negative values (which would indicate a data issue)
    red_negative = (test_df['red_fighter_days_since_last'] < 0).sum()
    blue_negative = (test_df['blue_fighter_days_since_last'] < 0).sum()
    print(f"Negative values check - Red: {red_negative}, Blue: {blue_negative}")
    
    return test_df

# Usage example:
test = enrich_test_data(df, test)

Processing 24 red stats and 24 blue stats
Calculating weighted trailing averages...


C:\Users\hegyi\AppData\Local\Temp\ipykernel_3144\2585840535.py:89: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  perf_df['weighted_trailing_avg'] = perf_df.groupby(['fighter', 'stat']).apply(


Mapping weighted trailing averages to test dataframe...
Calculating days since last fight for test set...
Negative values check - Red: 0, Blue: 0


In [57]:
# First, let's create a fight identifier and sort the dataframe in chronological order
df['fight_id'] = df.groupby(['event_date', 'red_fighter_name', 'blue_fighter_name']).ngroup()
df = df.sort_values(['event_date', 'fight_id'], ascending=[True, True])
df = df.reset_index(drop=True)

def calculate_fighter_stats(row, fighter_name):
    # Get all previous fights for this fighter (excluding current fight)
    current_fight_id = row['fight_id']
    current_date = row['event_date']
    
    mask = ((df['event_date'] < current_date) | 
            ((df['event_date'] == current_date) & (df['fight_id'] < current_fight_id))) & \
           ((df['red_fighter_name'] == fighter_name) | 
            (df['blue_fighter_name'] == fighter_name))
    
    # Get unique fights by taking the last round of each fight
    previous_fights = df[mask].groupby('fight_id').last().reset_index()
    
    if len(previous_fights) == 0:
        return {
            'avg_fight_time': 0,
            'defense': 0,
            'striking_accuracy': 0,
            'strikes_landed_per_min': 0,
            'strikes_absorbed_per_min': 0,
            'takedown_accuracy': 0,
            'takedown_defense': 0,
            'takedowns_per_15': 0,
            'submission_per_15': 0,
            'wins': 0,
            'losses': 0,
            'last_fight_days': None
        }
    
    stats = {}
    total_fights = len(previous_fights)
    
    # Calculate days since last fight
    last_fight_date = previous_fights['event_date'].max()
    stats['last_fight_days'] = (current_date - last_fight_date).days
    
    # Initialize counters
    total_time = 0
    total_strikes_landed = 0
    total_strikes_attempted = 0
    total_strikes_received = 0
    total_takedowns_landed = 0
    total_takedowns_attempted = 0
    total_takedowns_defended = 0
    total_takedowns_attempted_against = 0
    total_submissions = 0
    wins = 0
    
    # Process each previous fight
    for _, fight in previous_fights.iterrows():
        # Determine if fighter was red or blue in this fight
        is_red = fight['red_fighter_name'] == fighter_name
        prefix = 'red' if is_red else 'blue'
        opp_prefix = 'blue' if is_red else 'red'
        
        # Get all rounds for this fight
        fight_rounds = df[df['fight_id'] == fight['fight_id']]
        
        # Accumulate fight statistics across all rounds
        total_time += fight['final_time'] * len(fight_rounds)  # Multiply by number of rounds
        
        # Sum statistics across all rounds
        for _, round_data in fight_rounds.iterrows():
            total_strikes_landed += round_data[f'{prefix}_sig_str_landed']
            total_strikes_attempted += round_data[f'{prefix}_sig_str_attempted']
            total_strikes_received += round_data[f'{opp_prefix}_sig_str_landed']
            total_takedowns_landed += round_data[f'{prefix}_td_landed']
            total_takedowns_attempted += round_data[f'{prefix}_td_attempted']
            total_takedowns_defended += (round_data[f'{opp_prefix}_td_attempted'] - 
                                       round_data[f'{opp_prefix}_td_landed'])
            total_takedowns_attempted_against += round_data[f'{opp_prefix}_td_attempted']
            total_submissions += round_data[f'{prefix}_sub_att']
        
        # Count wins (only once per fight)
        if (is_red and fight['red_fighter_win']) or (not is_red and not fight['red_fighter_win']):
            wins += 1
    
    # Calculate averages
    total_minutes = total_time / 60
    
    stats['avg_fight_time'] = total_time / total_fights  # Now this is per fight, not per round
    stats['striking_accuracy'] = total_strikes_landed / total_strikes_attempted if total_strikes_attempted > 0 else 0
    stats['defense'] = 1 - (total_strikes_received / total_strikes_attempted) if total_strikes_attempted > 0 else 0
    stats['strikes_landed_per_min'] = total_strikes_landed / total_minutes if total_minutes > 0 else 0
    stats['strikes_absorbed_per_min'] = total_strikes_received / total_minutes if total_minutes > 0 else 0
    stats['takedown_accuracy'] = total_takedowns_landed / total_takedowns_attempted if total_takedowns_attempted > 0 else 0
    stats['takedown_defense'] = total_takedowns_defended / total_takedowns_attempted_against if total_takedowns_attempted_against > 0 else 0
    stats['takedowns_per_15'] = (total_takedowns_landed / total_minutes) * 15 if total_minutes > 0 else 0
    stats['submission_per_15'] = (total_submissions / total_minutes) * 15 if total_minutes > 0 else 0
    stats['wins'] = wins
    stats['losses'] = total_fights - wins
    
    return stats

print("Calculating trailing statistics...")

# Calculate trailing stats for each fighter in each fight
for prefix in ['red', 'blue']:
    # Calculate stats for each fight
    trailing_stats = df.apply(
        lambda row: calculate_fighter_stats(row, row[f'{prefix}_fighter_name']), 
        axis=1
    )
    
    # Convert the series of dictionaries to a DataFrame
    stats_df = pd.DataFrame(trailing_stats.tolist())
    
    # Add the columns to the main DataFrame with appropriate prefixes
    for col in stats_df.columns:
        df[f'{prefix}_fighter_{col}'] = stats_df[col]

Calculating trailing statistics...


to add:
-   add columns for both: man/women
-   weight is not in test =>> use max lbs of weeight class

# TODO
-   check hogy mivel több sor egy match hogyan számolja az átlagokat!

# Remove Columns

In [58]:
def clean_and_order_columns(dataframe):
    """
    Remove original performance columns that have been replaced with trailing averages,
    and order columns in a specific pattern: non-fighter columns first, then red fighter columns,
    then blue fighter columns (all in alphabetical order within their groups).
    
    Parameters:
    dataframe - DataFrame to clean and order (either training or test set)
    
    Returns:
    cleaned_df - DataFrame with only the necessary columns kept and ordered
    """
    import pandas as pd
    
    # Get current column list
    original_columns = dataframe.columns.tolist()
    columns_to_keep = []
    
    for col in original_columns:
        # Keep columns if they:
        # 1. Don't start with 'red_' or 'blue_', OR
        # 2. Start with 'red_fighter_' or 'blue_fighter_', OR
        # 3. End with '_trailing' or other trailing variants, OR
        # 4. Contain '_days_since_last'
        if (not (col.startswith('red_') or col.startswith('blue_'))) or \
           (col.startswith('red_fighter_') or col.startswith('blue_fighter_')) or \
           ('_trailing' in col) or \
           ('_days_since_last' in col):
            columns_to_keep.append(col)
    
    # Create new dataframe with only the columns to keep
    cleaned_df = dataframe[columns_to_keep]
    
    print(f"Original shape: {dataframe.shape}, Cleaned shape: {cleaned_df.shape}")
    print(f"Removed {dataframe.shape[1] - cleaned_df.shape[1]} columns")
    
    # Now order the columns according to the specified pattern
    # 1. Columns that don't start with 'red_' or 'blue_'
    # 2. Columns that start with 'red_' (in alphabetical order)
    # 3. Columns that start with 'blue_' (in alphabetical order)
    
    non_fighter_cols = [col for col in cleaned_df.columns if not (col.startswith('red_') or col.startswith('blue_'))]
    red_cols = [col for col in cleaned_df.columns if col.startswith('red_')]
    blue_cols = [col for col in cleaned_df.columns if col.startswith('blue_')]
    
    # Sort each group alphabetically
    non_fighter_cols.sort()
    red_cols.sort()
    blue_cols.sort()
    
    # Combine the ordered columns
    ordered_columns = non_fighter_cols + red_cols + blue_cols
    
    # Reorder the dataframe
    ordered_df = cleaned_df[ordered_columns]
    
    print(f"Column ordering: {len(non_fighter_cols)} non-fighter columns, "
          f"{len(red_cols)} red fighter columns, {len(blue_cols)} blue fighter columns")
    
    return ordered_df

# Usage examples:
df = clean_and_order_columns(df)  # Clean and order training set
test = clean_and_order_columns(test)  # Clean and order test set

Original shape: (18590, 144), Cleaned shape: (18590, 96)
Removed 48 columns
Column ordering: 7 non-fighter columns, 45 red fighter columns, 44 blue fighter columns
Original shape: (13, 86), Cleaned shape: (13, 86)
Removed 0 columns
Column ordering: 2 non-fighter columns, 42 red fighter columns, 42 blue fighter columns


In [59]:
def compare_dataframe_columns(df1, df2, df1_name="DataFrame 1", df2_name="DataFrame 2"):
    """
    Compare columns between two dataframes and report differences.
    
    Parameters:
    df1 - First DataFrame to compare
    df2 - Second DataFrame to compare
    df1_name - Name for the first DataFrame (for reporting)
    df2_name - Name for the second DataFrame (for reporting)
    
    Returns:
    comparison_dict - Dictionary containing comparison results
    """
    import pandas as pd
    
    # Get column sets
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)
    
    # Find common and unique columns
    common_cols = cols1.intersection(cols2)
    only_in_df1 = cols1 - cols2
    only_in_df2 = cols2 - cols1
    
    # Check data types for common columns
    dtype_mismatches = []
    for col in common_cols:
        if df1[col].dtype != df2[col].dtype:
            dtype_mismatches.append({
                'column': col,
                f'{df1_name}_dtype': df1[col].dtype,
                f'{df2_name}_dtype': df2[col].dtype
            })
    
    # Print summary
    print(f"Column Comparison: {df1_name} vs {df2_name}")
    print(f"Total columns in {df1_name}: {len(cols1)}")
    print(f"Total columns in {df2_name}: {len(cols2)}")
    print(f"Common columns: {len(common_cols)}")
    print(f"Columns only in {df1_name}: {len(only_in_df1)}")
    print(f"Columns only in {df2_name}: {len(only_in_df2)}")
    print(f"Data type mismatches in common columns: {len(dtype_mismatches)}")
    
    # Print detailed differences if any exist
    if only_in_df1:
        print(f"\nColumns only in {df1_name}:")
        for col in sorted(only_in_df1):
            print(f"  - {col}")
    
    if only_in_df2:
        print(f"\nColumns only in {df2_name}:")
        for col in sorted(only_in_df2):
            print(f"  - {col}")
    
    if dtype_mismatches:
        print("\nData type mismatches:")
        for mismatch in dtype_mismatches:
            print(f"  - {mismatch['column']}: {mismatch[f'{df1_name}_dtype']} vs {mismatch[f'{df2_name}_dtype']}")
    
    # Return comparison results as a dictionary
    comparison_dict = {
        'common_columns': list(common_cols),
        f'only_in_{df1_name}': list(only_in_df1),
        f'only_in_{df2_name}': list(only_in_df2),
        'dtype_mismatches': dtype_mismatches,
        'summary': {
            f'{df1_name}_column_count': len(cols1),
            f'{df2_name}_column_count': len(cols2),
            'common_column_count': len(common_cols),
            f'only_in_{df1_name}_count': len(only_in_df1),
            f'only_in_{df2_name}_count': len(only_in_df2),
            'dtype_mismatch_count': len(dtype_mismatches)
        }
    }
    
    return comparison_dict

# Usage example:
comparison = compare_dataframe_columns(df, test, df1_name="Training", df2_name="Test")

Column Comparison: Training vs Test
Total columns in Training: 96
Total columns in Test: 86
Common columns: 86
Columns only in Training: 10
Columns only in Test: 0
Data type mismatches in common columns: 0

Columns only in Training:
  - blue_fighter_age
  - blue_fighter_last_fight_days
  - fight_id
  - final_time
  - method
  - red_fighter_age
  - red_fighter_last_fight_days
  - red_fighter_win
  - round
  - time_format


In [63]:
df = df.drop(columns = ["blue_fighter_age",
                        "blue_fighter_last_fight_days",
                        "fight_id",
                        "final_time",
                        "method",
                        "red_fighter_age",
                        "red_fighter_last_fight_days",
                        "round",
                        "time_format"
                        ])

In [65]:
df = df.drop_duplicates()
df.shape

(13263, 87)

# Save

In [67]:
df.to_excel("data/train.xlsx", index=False)
test.to_excel("data/validation.xlsx", index = False)